In [ ]:
# !pip install plotnine

In [ ]:
# !pip install --upgrade numexpr

In [ ]:
# !pip install --upgrade bottleneck

In [ ]:
import pandas as pd
import numpy as np
import altair as alt
import pyarrow.parquet as pq
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.metrics import make_scorer, precision_score

from sklearn.metrics import confusion_matrix

from sklearn.pipeline import make_pipeline

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
# from plotnine.themes import theme_538

In [ ]:
def altair_conf_table(conf_df):
    '''
    Converts dataframe to format suitable for altair and generates a heat map
    -----------
    Parameters:
    conf_df: dataframe
    -----------
    Returns:
    a formatted altair heat map
    '''

    # Convert the seaborn dataframe to a long-form dataframe suitable for Altair
    conf_df_i_long = pd.melt(conf_df.reset_index(), id_vars='index')

    # Altair heatmap without legend and axis titles
    heatmap = alt.Chart(conf_df_i_long).mark_rect().encode(
        x='variable:O',
        y='index:O',
        color=alt.Color('value:Q', scale=alt.Scale(scheme='blues'),legend=None),  # Set color scale directly
        tooltip=['value:N']
    ).properties(
        width=400,
        height=300,
        title="Confusion Matrix"
    )

    # Text layer to display the values in each cell with increased font size
    text = alt.Chart(conf_df_i_long).mark_text(baseline='middle', fontSize=16).encode(
        x='variable:O',
        y='index:O',
        text='value:Q'
    )

    # Combine the heatmap and text layers
    alt_chart = heatmap + text

    # Remove the color legend
    alt_chart = alt_chart.configure_legend(
        strokeColor='white',
        fillColor='#f5f5f5',
        padding=10,
        cornerRadius=10,
        orient='none',
        legendX=650,
        legendY=0,
        direction='horizontal',
        labelFont='Arial',
        labelFontSize=12,
        labelColor='gray',
        title=None
    )

    # Remove axis titles
    alt_chart = alt_chart.configure_axis(
        labelFontSize=12,
        title=None,
        labelAngle=0 
    )
    

    # Show the Altair chart without legend and axis titles
    return  alt_chart

In [ ]:
warnings.filterwarnings("ignore")

# Data Collection

## RNAseq Expression Data

In [ ]:
# import gene RNAseq data file from a parquet formated file
table = pq.read_table('../data/processed/merged_df.parquet')

In [ ]:
# convert data table to a pandas data frame
merged_df = table.to_pandas()

In [ ]:
# delete table to free memory
del table

In [ ]:
# ispect the RNAseq dataframe
merged_df.head()

## Gene Information

In [ ]:
# gene information table
rows_genes = pd.read_csv('../data/external/rows-genes.csv')
rows_genes = rows_genes.set_index('gene_id')
rows_genes.index = rows_genes.index.astype(int)
rows_genes

# Data Preprocessing

The data was process and tables were merged on the notebook named "data_import_transformation.ipynb"

## Prepare data for model training

In [ ]:
# # assign X and y by selecting colunmns containing the gene features and predicted value
X = merged_df.iloc[:,:50281]
y = merged_df['act_demented']

## Split the Data

In [ ]:
# assign X and y by selecting colunmns containing the gene features and predicted value
X = merged_df.iloc[:,:50281]
y = merged_df['act_demented']

In [ ]:
# split the data into a training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Scale the Data

In [ ]:
# Standardize the data
scaler = StandardScaler()

# standarize training and test set
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Model Training

In [ ]:
# Create and train the logistic regression model
logreg_model = LogisticRegression(random_state=42)
logreg_model.fit(X_train_scaled, y_train)

## Model Evaluation

In [ ]:
pred_logreg = logreg_model.predict(X_test_scaled)
logreg_score = logreg_model.score(X_test_scaled, y_test)
print("Logistic regression: {:.2f}".format(logreg_score))

In [ ]:
conf_matrix = confusion_matrix(y_test, pred_logreg)
conf_df = pd.DataFrame(conf_matrix, index=['Actual Dementia', 'Actual No Dementia'],
                       columns=['Predicted Dementia', 'Predicted No Dementia'])

print(conf_df)

# Print a classification report
print("Classification Report:")
print(classification_report(y_test, pred_logreg))

altair_conf_table(conf_df)


In [ ]:
#Steph added
def combine_classification_rept(y_test, y_pred):
    """
    Input: y_test, y_pred parameters
    ---------
    Returns: dataframe of classification report
    
    """
    rept = classification_report(y_test, y_pred, output_dict=True)
    df_cls_rept = pd.DataFrame(rept).T
    df_cls_rept = df_cls_rept.round(2)
    
    #get key rows
    df_cls_rpt2 = df_cls_rept.iloc[:2,:]
    #df_cls_rept = df_cls_rept.sort_values(by=['f1-score'], ascending=False)
    return df_cls_rpt2

In [ ]:
#create table for lr w/o tuning
df_lr1 = combine_classification_rept(y_test, pred_logreg)
df_lr1['model'] = 'Logistical Regression - Default Parameters'
#df2 = df1.iloc[:2,:]
first_col = df_lr1.pop('model') 
df_lr1.insert(0, 'model', first_col) 
df_lr1

In [ ]:
#create table for lr
df_lr2 = combine_classification_rept(y_test, best_model_pred)
df_lr2['model'] = 'Logistical Regression - best C parameter'
first_col = df_lr2.pop('model') 
df_lr2.insert(0, 'model', first_col) 
df_lr2

### Regularization Strength (C)

In [ ]:
from sklearn.model_selection import GridSearchCV
# Define the logistic regression model
logreg_model = LogisticRegression()

# Define the hyperparameter grid for C
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

# Perform grid search with 5-fold cross-validation
grid_search = GridSearchCV(logreg_model, param_grid, cv=5, scoring='precision')
grid_search.fit(X_train_scaled, y_train)

# Print the best hyperparameter value
best_C = grid_search.best_params_['C']
print(f"Best C value: {best_C}")

# Evaluate the model with the best hyperparameter on the test set
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(X_test_scaled, y_test)
print(f"Accuracy on test set with best C: {test_accuracy}")

In [ ]:
best_model_pred = best_model.predict(X_test_scaled)
best_model_score = best_model.score(X_test_scaled, y_test)

conf_matrix = confusion_matrix(y_test, best_model_pred)
conf_df = pd.DataFrame(conf_matrix, index=['Actual Dementia', 'Actual No Dementia'],
                       columns=['Predicted Dementia', 'Predicted No Dementia'])

print(conf_df)

conf_df_i_long = pd.melt(conf_df.reset_index(), id_vars='index')

print(conf_df_i_long)

plt.style.use('ggplot')

# Print a classification report
print("Classification Report:")
print(classification_report(y_test, best_model_pred))

altair_conf_table(conf_df)


### Feature Selection

In [ ]:
model = best_model

In [ ]:
feature_importance = model.coef_[0]

In [ ]:
feature_names = X.columns
feature_importance_dict = dict(zip(feature_names, feature_importance))

In [ ]:
# Create a DataFrame or Series with feature names and their importance scores
feature_importance_df = pd.DataFrame(list(feature_importance_dict.items()), columns=['Feature', 'Importance'])
feature_importance_df

In [ ]:
# Set the 'Feature' column as the index
feature_importance_df.set_index('Feature', inplace=True)
feature_importance_df.index.name = 'gene_id'
feature_importance_df.index = feature_importance_df.index.astype(int)

# Display the feature importance DataFrame
print(feature_importance_df)

In [ ]:
rows_genes = pd.merge(rows_genes, feature_importance_df, left_index=True, right_index=True)

In [ ]:
rows_genes

In [ ]:
# Export complet set of genes
# rows_genes.to_csv(parent_directory+'/results/tables/all_genes_table.csv')
rows_genes.to_csv('../results/tables/all_genes_table.csv')

In [ ]:
# Select features
feature_importance_df = feature_importance_df.sort_values(by = 'Importance', ascending = False)
feature_importance_df

In [ ]:
# Plotting the histogram
plt.figure(figsize=(10, 6))
plt.hist(feature_importance_df['Importance'], bins=20, color='skyblue', edgecolor='black')
plt.title('Importance Histogram')
plt.xlabel('Importance Values')
plt.ylabel('Frequency')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

#### Feature Importance

Removing irrelevant or highly correlated features

In [ ]:
from sklearn.feature_selection import SelectFromModel

# set the number of features
max_features = 5000
selector = SelectFromModel(model, threshold=-np.inf, max_features=max_features)

In [ ]:
# how many columns at thies point

In [ ]:
X_train_scaled.shape

In [ ]:
selector.fit(X_train_scaled, y_train)

In [ ]:
# Get the boolean mask of selected features
feature_mask = selector.get_support()

In [ ]:
# Use boolean indexing to select the indexes
selected_indexes = feature_names[feature_mask]

In [ ]:
selected_indexes

In [ ]:
X_train_selected = selector.transform(X_train_scaled)
X_test_selected = selector.transform(X_test_scaled)

In [ ]:
model.fit(X_train_selected, y_train)

In [ ]:
%%time
# Perform RFE with cross-validation

# Specify the positive class label
positive_class_label = 'Dementia'

# Create a scorer for precision with the specified positive label
precision_scorer = make_scorer(precision_score, pos_label=positive_class_label)
cv = StratifiedKFold(n_splits=5)

# Use the precision_scorer in RFECV
rfe = RFECV(estimator=model, step=1, cv=cv, scoring=precision_scorer)

In [ ]:
%%time
rfe.fit(X_train_selected, y_train)

In [ ]:
# Get the optimal number of features
optimal_num_features = rfe.n_features_

In [ ]:
optimal_num_features

In [ ]:
len(rfe.support_)

In [ ]:
optimal_features_mask = rfe.support_

In [ ]:
optimal_features = selected_indexes[optimal_features_mask]

In [ ]:
# Get the list of selected features
optimal_features

In [ ]:
 # Save the array to a file
np.save('../results/plots/selected_features.npy', optimal_features)

In [ ]:
# Create a DataFrame to store the data
results_df = pd.DataFrame({
    'Number_of_Features': range(1, len(rfe.grid_scores_) + 1),
    'Cross_Validated_Score': rfe.grid_scores_
})

In [ ]:
results_df

In [ ]:
# Save feature importance externally
results_df.to_csv('../results/plots/rfe_cv_results.csv')

In [ ]:
# Print the recommendations
print(f"Optimal Number of Features: {optimal_num_features}")
# print(f"Selected Features: {list(selected_features)}")

#### Plot RFE from external file

In [ ]:
results_df_ext = pd.read_csv('../results/plots/rfe_cv_results.csv')

In [ ]:
results_df_ext.head()


__Recursive Feature Elimination with Cross-Validation__  
The plot (RFE-CV) shows the relationship between the number of selected features and the cross-validated score (accuracy in this case). Here's how to interpret the plot:

* X-axis (Number of Features Selected):  
Each point on the x-axis represents the number of features selected during the RFE process. It starts from 1 and goes up to the total number of features in your dataset.  
* Y-axis (Cross-Validated Score):  
The y-axis represents the cross-validated score (accuracy in this case) corresponding to the number of features selected. The higher the y-value, the better the model's performance.  

* Plot Shape:  
The elbow or plateau in the plot is the point where adding more features doesn't significantly improve the cross-validated score. We want to find a balance between the number of features and the model's performance.  


In [ ]:
# Assuming 'results_df_ext' is your DataFrame


alt.themes.enable("fivethirtyeight")

# Create Altair chart with attribute syntax
chart = alt.Chart(data=results_df_ext, width=600).mark_line(point=True)

chart = chart.encode(
    x=alt.X('Number_of_Features:O', 
            axis=alt.Axis(ticks=True, labels=True, 
                          values=list(range(0, max_features+1, 100)), 
                          title='Number of Features',
                          grid=True,  # Show gridlines
                          domain=False,  # Hide x-axis line
                          labelAlign='right',  # Align labels to the right for better readability
                          labelPadding=0,  # Add padding to labels
                          tickMinStep=100)  # Set the interval between ticks
           ),
    y=alt.Y('Cross_Validated_Score:Q',
           axis=alt.Axis(title='Cross Validation Score')),
    tooltip=['Number_of_Features:O', 'Cross_Validated_Score:Q']
)

chart = chart.properties(
    title='Recursive Feature Elimination with Cross-Validation (RFE-CV)'  # Add the title here
)

# Add a vertical red line to represent optimal_num_features
red_line = alt.Chart(pd.DataFrame({'x': [optimal_num_features]})).mark_rule(color='red').encode(x='x:O')

# Combine the main chart and the red line
combined_chart = chart + red_line

# Show the chart
combined_chart

#### Most important features

In [ ]:
# read selected features
optimal_features = np.load('../results/plots/selected_features.npy' , allow_pickle=True)

In [ ]:
# convert index key to integers for feature matching
optimal_features = optimal_features.astype(int)

In [ ]:
# merge selected features with gene information data
selected_rows = rows_genes [rows_genes .index.isin(optimal_features)]

In [ ]:
# inspect selected gene rows/features
selected_rows

In [ ]:
# Export selected genes table
selected_rows.to_csv('../results/tables/selected_genes.csv')

## Model performace with reduced selcted features

Let's compare how a model trained with all features performs when conpared with a model that was only trained with the selected features based on RFS

In [ ]:
X_train_prime = X_train[optimal_features.astype(str)]
X_test_prime = X_test[optimal_features.astype(str)]

In [ ]:
# Standardize the data
scaler = StandardScaler()

# standarize training and test set
X_train_i_scaled = scaler.fit_transform(X_train_prime)
X_test_i_scaled = scaler.transform(X_test_prime)

In [ ]:
# Create and train the logistic regression model
logreg_model_i = LogisticRegression(random_state=42 ,C=0.001 )

In [ ]:
logreg_model_i.fit(X_train_i_scaled, y_train)

pred_logreg_i = logreg_model_i.predict(X_test_i_scaled)
logreg_score_i = logreg_model_i.score(X_test_i_scaled, y_test)
print("Logistic regression: {:.2f}".format(logreg_score_i))

In [ ]:
conf_matrix_i = confusion_matrix(y_test, pred_logreg_i)
conf_df_i = pd.DataFrame(conf_matrix_i, index=['Actual Dementia', 'Actual No Dementia'],
                       columns=['Predicted Dementia', 'Predicted No Dementia'])

print(conf_df_i)

# Print a classification report
print("Classification Report:")
print(classification_report(y_test, pred_logreg_i))

alt.themes.enable("default")
altair_conf_table(conf_df_i)


In [ ]:
#create table for lr
df_lr3 = combine_classification_rept(y_test, pred_logreg_i)
df_lr3['model'] = "Logistical regression - reduced features"
#df2 = df1.iloc[:2,:]
first_col = df_lr3.pop('model') 
df_lr3.insert(0, 'model', first_col) 
df_lr3

In [ ]:
df_combined =pd.concat([df_lr1,df_lr2,df_lr3])
df_combined

### Solver

In [ ]:
# ### Solver

# Define the solvers you want to compare
solvers = ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']

# Initialize an empty dictionary to store the mean cross-validation scores for each solver
mean_scores = {}

# Loop through each solver and calculate the mean cross-validation score
for solver in solvers:
    model = LogisticRegression(random_state=42,C=0.001,solver=solver)
                               
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')  # You can use other scoring metrics as well
    mean_scores[solver] = np.mean(scores)

In [ ]:
# Print mean scores for each solver
for solver, mean_score in mean_scores.items():
    print(f'{solver}: {mean_score}')

# Choose the solver with the highest mean score
best_solver = max(mean_scores, key=mean_scores.get)
print(f'Best Solver: {best_solver}')

# Model

In [ ]:
logreg_model = LogisticRegression(random_state=42 ,C=0.001, solver='lbfgs' )

In [ ]:
logreg_model.fit(X_train_i_scaled, y_train)

## Overfitting

When working with an small data set we need to awared that we do not ovorfit ther model. In such a case the model may learn the noise in the data rather than the actual patterns on the data. In order to check for overfitting we used cross-validation to det4ermone the model's performance on multiple train-test splits. If the model consistently performs well across different subsets of the data, it's a positive sign that the model's performance is not dependent on a specific random split of the data.

In [ ]:
# Define the logistic regression model with chosen parameters
logreg_model = LogisticRegression(random_state=42, C=0.001, solver='lbfgs',
                                  max_iter=1000)

In [ ]:
# Create a pipeline with standard scaling (if applicable) and the model
model_pipeline = make_pipeline(StandardScaler(), logreg_model)

In [ ]:
# Cross-validation strategy
cv = StratifiedKFold(n_splits=12, shuffle=True, random_state=42)

In [ ]:
# Filter out FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import warnings

# Ignore FutureWarnings and all warnings
warnings.filterwarnings("ignore")

# Perform cross-validation and get the scores
train_scores = cross_val_score(model_pipeline, X, y,
                               cv=cv, scoring='accuracy', n_jobs=-1)

test_scores = cross_val_score(logreg_model, X, y,
                              cv=cv, scoring='accuracy', n_jobs=-1)


In [ ]:
# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Fold': np.arange(1, len(train_scores) + 1),
    'Train Accuracy': train_scores,
    'Test Accuracy': test_scores
})

results_df

In [ ]:
# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(results_df['Fold'], results_df['Train Accuracy'], marker='o', label='Train Accuracy')
plt.plot(results_df['Fold'], results_df['Test Accuracy'], marker='o', label='Test Accuracy')
plt.title('Cross-Validation Results')
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

In general, the training accuracies are reasonably high wich indicats that the model is learning from the training data. However, there is some variability in the accuracies across the different folds.
The test accuracies are lower than the training accuracies which may be expected however the differences in accuracy across the various folds might indicate some level of overfitting.

## Features

In [ ]:
selected_rows

## Discusion

Regarding long-term care insurance underwriting, our primary goal is to enhance the accuracy of predicting No Dementia in patients while minimizing false positives. The original model, which uses over 50,000 features, has a greater precision of 0.85 for No Dementia, indicating that 85% of patients identified as No Dementia are accurate. In contrast, the new model has a precision of approximately 0.76 for No Dementia, meaning that around 76% of instances predicted as No Dementia are true negatives.
Although the new model correctly identifies more true negatives (35 out of 41) than the original model (34 out of 41), the precision metric considers the ratio of true negatives to false positives. The original model achieves a higher precision in predicting No Dementia, indicating a lower occurrence of false positives.
It's important to acknowledge that despite utilizing only 283 features or the expression of 283 genes, the new model achieves a comparable specificity for predicting No Dementia. This reduction in feature dimensionality has practical implications, such as reduced computational costs and improved model interpretability.
Furthermore, the possibility of obtaining gene expression data on 283 targeted genes using faster chip technology, such as customized microarrays or targeted RNA sequencing panels, is worth investigating. These technologies allow for high-throughput analysis, offering opportunities for more efficient and streamlined processes.

## Libraries

In [ ]:
import pkg_resources

def get_library_versions():
    """
    input list of library names as str
    ----------
    return dictionary with k,v as library,version
    """
    # List of libraries used in the project
    libraries = ['pandas', 'numpy', 'altair', 'pyarrow', 'scikit-learn', 'seaborn', 'matplotlib', 'warnings']

    # Dictionary to store library versions
    library_versions = {}

    # Iterate through the list of libraries and get their versions
    for library in libraries:
        try:
            version = pkg_resources.get_distribution(library).version
            library_versions[library] = version
        except pkg_resources.DistributionNotFound:
            library_versions[library] = "Not Installed"

    return library_versions

if __name__ == "__main__":
    versions = get_library_versions()

    # Print the library versions
    print("Library Versions:")
    for library, version in versions.items():
        print(f"{library}: {version}")